## 수치미분과 딥러닝을 이용한 당뇨병 예측  

In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

## Diabetes 클래스 정의

In [2]:
# Diabetes Class

class Diabetes:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        
        # 2층 hidden layer unit 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_nodes)  
        self.b2 = np.random.rand(hidden_nodes)
        
        # 3층 output layer unit : 1 개 
        self.W3 = np.random.rand(hidden_nodes,output_nodes)
        self.b3 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        print("Diabetes object is created !!!")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, input_data):    
        
        z2 = np.dot(input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    
    def accuracy(self, input_data, target_data):
        
        matched_list = []
        not_matched_list = []
        
        # list which contains (index, label, prediction) value
        index_label_prediction_list = []
        
        # temp list which contains label and prediction in sequence
        temp_list = []
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
                temp_list.append(index)
                temp_list.append(target_data[index])
                temp_list.append(logical_val)
                
                index_label_prediction_list.append(temp_list)
                
                temp_list = []
                
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list, index_label_prediction_list
    
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)

## csv 파일로부터 training_data,  test_data 분리

In [7]:
# 데이터 불러오기

loaded_data = np.loadtxt('./diabetes.csv', delimiter=',', dtype=np.float32)

print("loaded_data.shape = ", loaded_data.shape)

# 임시 저장 리스트
training_data_list = []
test_data_list = []

# 전체 30 % 라는것은 전체 길이에 대해 3 으로 나누어주면 됨
seperation_rate = 3

for index in range(len(loaded_data)):
    
    if (index % seperation_rate) == 0:
        
        test_data_list.append(loaded_data[index])
        
    else:
        
        training_data_list.append(loaded_data[index])
        
# generate training data from training_data_list using np.arrya(...)
training_data = np.array(training_data_list)

# generate test data from test_data_list using np.arrya(...)
test_data = np.array(test_data_list)

# verification shape
print("training_data.shape = ", training_data.shape)
print("test_data.shape = ", test_data.shape)

# save training & test data (.csv)
np.savetxt('./diabetes_generated_training_data.csv', training_data, delimiter=',')
np.savetxt('./diabetes_generated_test_data.csv', test_data, delimiter=',')

loaded_data.shape =  (759, 9)
training_data.shape =  (506, 9)
test_data.shape =  (253, 9)


## Hyper-Parameter 설정 및 train 실행

In [8]:
#hyper-parameter
i_nodes = training_data.shape[1] - 1    # input nodes 개수
h1_nodes = 30  # hidden nodes 개수
o_nodes = 1    # output nodes 개수
lr = 1e-2      # learning rate
epochs = 10   # 반복횟수

# Diabetes 객체 생성
obj = Diabetes(i_nodes, h1_nodes, o_nodes, lr)

print("Neural Network Learning using Numerical Derivative...")

start_time = datetime.now()

for step in range(epochs):
    
    for index in range(len(training_data)):
        
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        obj.train(input_data, target_data)
        
    if (step % 2 == 0):
        print("epochs = ", step, "loss value = ", obj.loss_val())

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

Diabetes object is created !!!
Neural Network Learning using Numerical Derivative...
epochs =  0 loss value =  0.6563382666668434
epochs =  2 loss value =  0.48464630150573307
epochs =  4 loss value =  0.37276311757806707
epochs =  6 loss value =  0.3012594180691766
epochs =  8 loss value =  0.25439157269002066

Elapsed Time =>  0:01:30.040000


## Accuracy 검증

In [9]:
test_input_data = test_data[ :, 0:-1 ]
test_target_data = test_data[ :, -1 ]

(true_list, false_list, index_label_prediction_list) = obj.accuracy(test_input_data, test_target_data)

Accuracy =>  0.7351778656126482


In [10]:
print(index_label_prediction_list)

[[2, 0.0, 1], [4, 1.0, 0], [6, 0.0, 1], [9, 1.0, 0], [12, 0.0, 1], [13, 1.0, 0], [14, 1.0, 0], [28, 1.0, 0], [36, 0.0, 1], [38, 0.0, 1], [41, 0.0, 1], [42, 0.0, 1], [56, 0.0, 1], [62, 0.0, 1], [65, 0.0, 1], [66, 1.0, 0], [70, 1.0, 0], [72, 0.0, 1], [75, 0.0, 1], [80, 0.0, 1], [81, 1.0, 0], [84, 0.0, 1], [88, 1.0, 0], [90, 1.0, 0], [94, 0.0, 1], [96, 0.0, 1], [97, 0.0, 1], [98, 1.0, 0], [101, 0.0, 1], [107, 0.0, 1], [110, 1.0, 0], [113, 1.0, 0], [117, 0.0, 1], [119, 1.0, 0], [122, 1.0, 0], [127, 0.0, 1], [132, 0.0, 1], [141, 0.0, 1], [146, 0.0, 1], [149, 1.0, 0], [150, 1.0, 0], [154, 1.0, 0], [156, 1.0, 0], [158, 1.0, 0], [160, 1.0, 0], [162, 0.0, 1], [164, 1.0, 0], [165, 0.0, 1], [176, 1.0, 0], [177, 0.0, 1], [183, 1.0, 0], [187, 0.0, 1], [190, 1.0, 0], [200, 1.0, 0], [201, 0.0, 1], [206, 1.0, 0], [210, 0.0, 1], [217, 1.0, 0], [220, 1.0, 0], [223, 0.0, 1], [225, 1.0, 0], [227, 1.0, 0], [229, 0.0, 1], [230, 1.0, 0], [241, 0.0, 1], [245, 1.0, 0], [249, 1.0, 0]]


## 생성해서 저장한 
## diabetes_generated_training_data / diabetes_generated_test_data 파일로 딥러닝 수행

In [14]:
generated_training_data = np.loadtxt('./diabetes_generated_training_data.csv', delimiter=',',  dtype=np.float32)
generated_test_data = np.loadtxt('./diabetes_generated_test_data.csv', delimiter=',',  dtype=np.float32)

print("generated_training_data.shape = ", generated_training_data.shape)
print("generated_test_data.shape = ", generated_test_data.shape)

generated_training_data.shape =  (506, 9)
generated_test_data.shape =  (253, 9)


In [15]:
#hyper-parameter
i_nodes = generated_training_data.shape[1] - 1    # input nodes 개수
h1_nodes = 30  # hidden nodes 개수
o_nodes = 1    # output nodes 개수
lr = 1e-2      # learning rate
epochs = 10   # 반복횟수

# Diabetes 객체 생성
obj = Diabetes(i_nodes, h1_nodes, o_nodes, lr)

print("Neural Network Learning using Numerical Derivative...")

start_time = datetime.now()

for step in range(epochs):
    
    for index in range(len(generated_training_data)):
        
        input_data = generated_training_data[index, 0:-1]  
        target_data = generated_training_data[index, [-1]] 
        
        obj.train(input_data, target_data)
        
    if (step % 2 == 0):
        print("epochs = ", step, "loss value = ", obj.loss_val())

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

Diabetes object is created !!!
Neural Network Learning using Numerical Derivative...
epochs =  0 loss value =  0.7246342001262254
epochs =  2 loss value =  0.5241855803124901
epochs =  4 loss value =  0.39268151184965433
epochs =  6 loss value =  0.3107816876436779
epochs =  8 loss value =  0.2589951376978359

Elapsed Time =>  0:01:37.225000


In [16]:
test_input_data = generated_test_data[ :, 0:-1 ]
test_target_data = generated_test_data[ :, -1 ]

(true_list, false_list, index_label_prediction_list) = obj.accuracy(test_input_data, test_target_data)

Accuracy =>  0.7549407114624506


In [17]:
print(index_label_prediction_list)

[[2, 0.0, 1], [4, 1.0, 0], [6, 0.0, 1], [12, 0.0, 1], [13, 1.0, 0], [14, 1.0, 0], [28, 1.0, 0], [36, 0.0, 1], [38, 0.0, 1], [41, 0.0, 1], [42, 0.0, 1], [56, 0.0, 1], [62, 0.0, 1], [65, 0.0, 1], [66, 1.0, 0], [70, 1.0, 0], [72, 0.0, 1], [80, 0.0, 1], [81, 1.0, 0], [84, 0.0, 1], [88, 1.0, 0], [90, 1.0, 0], [94, 0.0, 1], [96, 0.0, 1], [97, 0.0, 1], [98, 1.0, 0], [101, 0.0, 1], [107, 0.0, 1], [110, 1.0, 0], [113, 1.0, 0], [117, 0.0, 1], [119, 1.0, 0], [122, 1.0, 0], [127, 0.0, 1], [132, 0.0, 1], [141, 0.0, 1], [146, 0.0, 1], [149, 1.0, 0], [150, 1.0, 0], [154, 1.0, 0], [156, 1.0, 0], [160, 1.0, 0], [162, 0.0, 1], [164, 1.0, 0], [177, 0.0, 1], [183, 1.0, 0], [187, 0.0, 1], [190, 1.0, 0], [200, 1.0, 0], [201, 0.0, 1], [206, 1.0, 0], [210, 0.0, 1], [217, 1.0, 0], [223, 0.0, 1], [225, 1.0, 0], [227, 1.0, 0], [228, 0.0, 1], [229, 0.0, 1], [230, 1.0, 0], [241, 0.0, 1], [245, 1.0, 0], [249, 1.0, 0]]
